In [2]:
print("spark running")

In [3]:
# configure spark for cluster set up
%%configure -f
{"executorMemory": "6G", "executorCores": 8, "numExecutors":4}

SyntaxError: invalid syntax (<ipython-input-3-ef04e31e168e>, line 2)

In [4]:
def riskRateRetrieval():
    rates = []
    date = datetime.datetime.now().date() - timedelta(1)
    mydata = quandl.get("USTREASURY/YIELD", start_date= '2018-03-09', authtoken="ML7Kam1GdfzQ3Y4Ms-Ki")
    riskFreeRate = [mydata["3 MO"]]
    for i in range(1,4):
        if i % 2 == 0:
            riskFreeRate.append(riskFreeRate[i-1] + (riskFreeRate[i-1] * (0.125 * i)))
        else:
            riskFreeRate.append(riskFreeRate[i-1] - (riskFreeRate[i-1] * (0.125 * i)))
    for rate in riskFreeRate:
        rates.append(float(rate[0]))
    return rates


In [5]:
def getSNPList():
    url = 'https://dimon.ca/api/snp500'  # Set destination URL here
    sessionID = getSessionID()
    print("SessiosessionID = ", sessionID)  # Comment if on cloud
    post_fields = {'session': sessionID}     # Set POST fields here

    request = Request(url, urlencode(post_fields).encode())
    data = urlopen(request).read().decode()
    data = json.loads(data)
    return data


def getSessionID():
    urllib.request.urlcleanup()  # removes any cache, cookies etc
    link = 'https://dimon.ca/snp500/'  # Set destination URL here
    id = ""
    data = urllib.request.urlopen(link)
    data = BeautifulSoup(data, "html.parser")
    id = data.find('input', attrs={'name': 'session'}).get('value')
    return id




In [6]:
def retrieveYahooData(ticker):
    try:
        if "." in ticker:  # some tickers in list have "." when not needed
            ticker = ticker.replace(".", "")  # Removing "."
        url = createYahooUrlWithDate(ticker)
        if url:
            data = urlopen(url)
            data = json.loads(data.read().decode())
            return data
        return False # false if url is false
    except urllib.error.HTTPError as err:
        if err.code == 404:
            print("Page not found!")
            return False
        elif err.code == 403:
            print("Access denied!")
            return False
        else:
            print("Something happened! Error code", err.code)
            return False
    except urllib.error.URLError as err:
        print("Some other error happened:", err.reason)
        return False


def createYahooUrlWithDate(optionTicker):
    url = "https://query2.finance.yahoo.com/v7/finance/options/" + optionTicker
    data = urlopen(url)
    data = json.loads(data.read().decode())
    expirationDates = data['optionChain']['result'][0]['expirationDates']
    expDate = 0
    if expirationDates:
        for item in expirationDates:
            dt = datetime.datetime.fromtimestamp(item) - datetime.datetime.now()
            if dt.days > 0:  # should run the day before but is seen as 0 days and number of hours
                expDate = item
                break
        return url + "?date=" + str(expDate)
    return False # false if no expiration dates hence no options

In [8]:
# Dictionaries for json output
option_prices = {}
results = {}
call_results = {}
put_results = {}


def optionCalculaionSpark(ticker, riskFreeRates):
    print(ticker)
    option_type = "not set"
    strike_price = 0        # S(T) price at maturity
    current_value = 0           # S(0) spot price, price of stock now
    volatility = .3672                  # sigma i.e. volatility of underlying stock
    expires = 55  # Number of days until maturity date

    risk_free_rate = riskFreeRates # risk free rate from fed

    data = retrieveYahooData(ticker)
    if "regularMarketPrice" in data['optionChain']['result'][0]['quote']:
        current_value = data['optionChain']['result'][0]['quote']['regularMarketPrice']
        data = data['optionChain']['result'][0]['options']
        for option in data:
            calls, puts = option['calls'], option['puts']
        # Assigning variables for calc and running sim
        for call in calls:
            option_type = "Call"
            strike_price = call['strike']           # S(T) price at maturity
            results['StrikePrice'] = strike_price 
            volatility = call['impliedVolatility']
            results['Volatility'] = volatility
            dt = datetime.datetime.fromtimestamp(
            call['expiration']) - datetime.datetime.now()
            expires = dt.days+ 1 # doesnt run on the exp date so plus 1 to account for that
            option_prices['NumberOfDays'] = expires
            option_prices['ExpirationDate'] = datetime.datetime.fromtimestamp(
            call['expiration']).strftime('%Y-%m-%d')
            for rate in risk_free_rate:
                results['RiskFreeRate'] = rate
                runSimulaion(option_type, strike_price, current_value,
                                volatility, rate, expires, ticker)
                results[option_type] = call_results.copy()
                priceResults.append(results.copy())
        results = {}
        for put in puts:
            option_type = "Put"
            strike_price = put['strike']            # S(T) price at maturity
            results['StrikePrice'] = strike_price 
            volatility = put['impliedVolatility']
            results['Volatility'] = volatility
            dt = datetime.datetime.fromtimestamp(
            put['expiration']) - datetime.datetime.now()
            expires = dt.days + 1 # doesnt run on the exp date so plus 1 to account for that
            option_prices['NumberOfDays'] = expires
            for rate in risk_free_rate:
                results['RiskFreeRate'] = rate
                runSimulaion(option_type, strike_price, current_value,
                                volatility, rate, expires, ticker)
                results[option_type] = put_results.copy()
                priceResults.append(results.copy())
        option_prices['Prices'] = priceResults
    else:
        call_results['NA'] = "MISSING DATA"
        put_results['NA'] = "MISSING DATA"
        results[option_type] = call_results
        results[option_type] = put_results
        option_prices['Prices'] = results

    with open('optionPrices.json', 'w') as outfile:
        json.dump(option_prices, outfile)
    #writeResultHive()


def runSimulaion(option_type, strike_price, current_value, volatility, risk_free_rate, expires, ticker):
    start_date = datetime.date.today()
    num_simulations = 10000
    option_prices = []
    for x in range(0, 5):
        # W(T) Wiener process/Brownian motion  = math.sqrt(T) * random.gauss(0, 1.0)
        # sequential approach, calculate option price every day until expiry

        times = []
        # 1 .. expires inclusive
        for i in range(1, expires + 1):
            # results from each simulation step
            sim_results = []
            T = i / 365               # days in the future
            times.append(i)
            for rate in risk_free_rate:
                results['RiskFreeRate'] = rate
                for j in range(num_simulations):
                    sim_results.append(sim_option_price(time.time() + j, current_value,
                                                        rate, volatility, T, strike_price, option_type))
                # e to the power of ()
                discount_factor = math.exp(-rate * T)
                option_prices.append(
                    discount_factor * (sum(sim_results) / float(num_simulations)))
                if option_type == "Call":
                    call_results[(
                        str(start_date + datetime.timedelta(days=i)))] = option_prices[i - 1]
                    # print(ticker, " ", option_type, " ", "Option Price ",
                    # option_prices[i - 1], " at ", start_date + datetime.timedelta(days=i))
                else:
                    put_results[(
                        str(start_date + datetime.timedelta(days=i)))] = option_prices[i - 1]
                    # print(ticker, " ", option_type, " ", "Option Price ",
                    # option_prices[i - 1], " at ", start_date + datetime.timedelta(days=i))


def call_payoff(asset_price, strike_price):
    return max(0.0, asset_price - strike_price)


def put_payoff(asset_price, strike_price):
    return max(0.0, strike_price - asset_price)

# simulate the option price
def sim_option_price(seed, current_value, risk_free_rate, volatility, T, strike_price, option_type):
    random.seed(seed)
    asset_price = current_value * \
        math.exp((risk_free_rate - .5 * volatility**2) * T +
                 volatility * math.sqrt(T) * random.gauss(0, 1.0))
    if option_type == "Call":
        return call_payoff(asset_price, strike_price)
    else:
        return put_payoff(asset_price, strike_price)

In [9]:
def printDescription():
	print("Program to retrive and print the ticker symbols of stocks in S and P 500 list")

In [10]:
import json
import datetime
import random
import math
import time
import urllib
from urllib.request import urlopen
from bs4 import BeautifulSoup 
from urllib.parse import urlencode
from urllib.request import Request, urlopen
from datetime import timedelta
import quandl

symbols = []
start_time = time.time()
printDescription()
data = getSNPList()
rates = riskRateRetrieval()
for item in data['members']:
	symbols.append(item['sym'])

#optionCalculaionSpark('MMM', rates)
symbols = ['MMM']
symbols = spark.sparkContext.parallelize(symbols)
result = symbols.map(lambda sym: optionCalculaionSpark(sym, rates))
result.collect()
#print(result.collect())
result.write.save('E:\ProjectDB', format='json', mode='append')
print("******** finsihed in %s seconds ********" % (time.time() -start_time))

Program to retrive and print the ticker symbols of stocks in S and P 500 list
SessiosessionID =  600-6-cebafccdcfcea


NameError: name 'spark' is not defined